In [1]:
!pip install redis

In [2]:
import redis
import json

In [3]:
r = redis.Redis(
    host='redis',
    port=6379,
    db=0,
    decode_responses=True
)

In [36]:
actual_id = 1

def next_id():
    global actual_id
    id = str(actual_id)
    actual_id = actual_id + 1
    return id

In [ ]:
def addplayer(id, name, country, score=0):
    player_key = f"player:{id}"

    player_data = {
        "name": name,
        "country": country,
        "games_played": 0,
        "score": score
    }

    r.hset(player_key, mapping=player_data)
    r.zadd("leaderboard", {id: score})

In [45]:
nom = input("Nombre: ")
pais = input("Pais: ")
id = next_id()

addplayer(id, nom, pais)

In [37]:
def updatescore(id, points):

    player_key = f"player:{id}"

    if not r.exists(player_key):
        print(f"Error: Jugador con ID {id} no encontrado.")
        return

    games_played = r.hget(player_key, "games_played")
    games_played = int(games_played) + 1
    r.hset(player_key, "games_played", games_played)

    score = r.hget(player_key, "score")
    new_score = int(score) + points
    r.hset(player_key, "score", new_score)

    r.zadd("leaderboard", {id:new_score})

In [ ]:
updatescore(1, 10)
updatescore(1, 15)
updatescore(2, 15)

In [ ]:
def playerinfo(id):
    player_key = f"player:{id}"

    info = r.hgetall(player_key)

    if not info:
        print(f"Error: Jugador con ID {id} no encontrado.")
        return
    
    for key, value in info.items():
        print(f"  > {key.capitalize()}: {value}")
    return info

In [40]:
playerinfo(1)

  >Name: aa
  >Country: aa
  >Games_played: 0
  >Score: 0


{'name': 'aa', 'country': 'aa', 'games_played': '0', 'score': '0'}

In [42]:
def showtopplayers(n):

    top_players = r.zrevrange("leaderboard", 0, n -1, withscores=True)

    if not top_players:
        print("El ranking esta vacio")
        return
    
    for i, (id, score) in enumerate(top_players):

        player_key = f"player:{id}"
        
        res = r.hgetall(player_key)

        print(f" #{i+1}: { res['name']} ({res['country']})")
        print(f"     ID: {id} | Puntuación: {int(res['score'])}")
        print("  " + "-"*20)

In [46]:
showtopplayers(2)

 #1: dd (dd)
     ID: 4 | Puntuación: 0
  --------------------
 #2: cc (cc)
     ID: 3 | Puntuación: 0
  --------------------


In [ ]:
import datetime as date

def get_date_key(date_obj):
    return f"unique:players:{date_obj.isoformat()}"


def register_login(player_id):

    today_key = get_date_key(date.today())

    changed = r.pfadd(today_key, player_id)

    if changed:
        print(f"[Login] {player_id} registrado por primera vez hoy")
    else
        print(f"[Login] {player_id} ya habia iniciado sesicon")
        

In [ ]:
def count_unique_login(date_str):
    key = f"unique:players:{date_str}"
    count = r.pfcount(key)

    print(f"\n[Reporte] Jugadores unico (aprox) el {date_str: {count}}")

In [ ]:
def weekley_report(dates_list):
    keys_to_merge = [f"unique:players:{d}" for d in dates_list]

    report_key = "unique:players:weekly_report" # Clave temporal

    r.pfmerge(report_key, *keys_to_merge)

    total_unique = r.pfcount(report_key)

    print(f"\n--- Reporte Semanal (Dias: {len(dates_list)}) ---")
    print(f"Total de jugadores unicos (aprox): {total_unique}")

    # Eliminamos la clave temporal
    r.delete(report_key)
    
    return total_unique

In [ ]:
weekley_report(["2023-11-10", "2025-11-11"])